In [1]:
import pandas as pd
import numpy as np
from os import listdir

In [2]:
folder = "cleaned_power_5yr"
print(listdir(folder))
onlyfiles = [f"{folder}/{f}" for f in listdir(folder)]

['cleaned_power_16.csv', 'cleaned_power_17.csv', 'cleaned_power_18.csv', 'cleaned_power_19.csv', 'cleaned_power_20.csv']


In [3]:
weather = pd.read_csv("weather_5yr.csv")
#power = pd.read_csv("cleaned_power.csv")
yr_power = []
for f in onlyfiles:
    yr_power.append(pd.read_csv(f).dropna())
power = pd.concat(yr_power, ignore_index=True)

C:\Users\Wade\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (41,42,43,49,51,56,73,75,89) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Wade\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Wade\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Wade\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9) have mixed types.Specify dtype option on import or set low_memory=False.
  interact

### Convert all dates to datetime object

In [4]:
#power["DATE"] = power.apply(lambda x: pd.Timestamp(x["Timestamp"]), axis=1)
#weather["DATE"] = weather.apply(lambda x: pd.Timestamp(x["DATE"]), axis=1)
power["DATE"] = pd.to_datetime(power["Timestamp"])
weather["DATE"] = pd.to_datetime(weather["DATE"])

### Ensure total power is included, reduce to match weather interval

In [5]:
#Trim power to match date range
date_mismatch = False
if date_mismatch:
    cutoff = power.index[power['DATE'] == pd.Timestamp('3/1/2020 0:00')].tolist()
    power = power.iloc[:cutoff+1,:]

In [6]:
power

,Timestamp,Chiller-2 Power,Chiller-3 Power,Chiller-4 Power,Chiller-5 Power,Chiller-6 Power,Chiller-7 Power,Chiller-8 Power,Chiller-9 Power,Chiller-10 Power,DATE
0,01-Mar-16 00:00:00,0,0,1179.439331,0,0,15.89638424,0,0.102776416,0,2016-03-01 00:00:00
1,01-Mar-16 00:01:00,0,0,1195.945435,0,0,15.06277466,0,0.098978162,0,2016-03-01 00:01:00
2,01-Mar-16 00:02:00,0,0,1175.737549,0,0,15.89653492,0,0.028593669,0,2016-03-01 00:02:00
3,01-Mar-16 00:03:00,0,0,1179.98291,0,0,15.89653492,0,0.10939043,0,2016-03-01 00:03:00
4,01-Mar-16 00:04:00,0,0,1198.777954,0,0,15.06291771,0,0.098978162,0,2016-03-01 00:04:00
...,...,...,...,...,...,...,...,...,...,...,...
2629435,28-Feb-21 23:55:00,0,1382.180176,0,0,0,0,0,-0.003145,0.210209,2021-02-28 23:55:00
2629436,28-Feb-21 23:56:00,0,1367.581299,0,0,0,0,0,-0.002276,0.0,2021-02-28 23:56:00
2629437,28-Feb-21 23:57:00,0,1352.855835,0,0,0,0,0,-0.003423,0.0,2021-02-28 23:57:00
2629438,28-Feb-21 23:58:00,0,1352.390137,0,0,0,0,0,0.0,0.0,2021-02-28 23:58:00


In [7]:
# ID power columns
power_cols = [
        'Chiller-2 Power',
        'Chiller-3 Power',
        'Chiller-4 Power',
        'Chiller-5 Power',
        'Chiller-6 Power',
        'Chiller-7 Power',
        'Chiller-8 Power',
        'Chiller-9 Power',
        'Chiller-10 Power']

In [8]:
# Replace string errors in power values with 0
for col in power_cols:
    power[col] = pd.to_numeric(power[col], errors='coerce')
# force all values less than 100 to 0
power[power[power_cols]<100]=np.nan
# They are pushed to NaN first, then replaced with 0
power = power.replace(np.nan,0)

In [9]:
# Add total power column
if "Total Power" not in power.columns:
    power["Total Power"] = power[power_cols].sum(axis=1)

In [10]:
power

,Timestamp,Chiller-2 Power,Chiller-3 Power,Chiller-4 Power,Chiller-5 Power,Chiller-6 Power,Chiller-7 Power,Chiller-8 Power,Chiller-9 Power,Chiller-10 Power,DATE,Total Power
0,01-Mar-16 00:00:00,0.0,0.000000,1179.439331,0.0,0.0,0.0,0.0,0.0,0.0,2016-03-01 00:00:00,1179.439331
1,01-Mar-16 00:01:00,0.0,0.000000,1195.945435,0.0,0.0,0.0,0.0,0.0,0.0,2016-03-01 00:01:00,1195.945435
2,01-Mar-16 00:02:00,0.0,0.000000,1175.737549,0.0,0.0,0.0,0.0,0.0,0.0,2016-03-01 00:02:00,1175.737549
3,01-Mar-16 00:03:00,0.0,0.000000,1179.982910,0.0,0.0,0.0,0.0,0.0,0.0,2016-03-01 00:03:00,1179.982910
4,01-Mar-16 00:04:00,0.0,0.000000,1198.777954,0.0,0.0,0.0,0.0,0.0,0.0,2016-03-01 00:04:00,1198.777954
...,...,...,...,...,...,...,...,...,...,...,...,...
2629435,28-Feb-21 23:55:00,0.0,1382.180176,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2021-02-28 23:55:00,1382.180176
2629436,28-Feb-21 23:56:00,0.0,1367.581299,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2021-02-28 23:56:00,1367.581299
2629437,28-Feb-21 23:57:00,0.0,1352.855835,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2021-02-28 23:57:00,1352.855835
2629438,28-Feb-21 23:58:00,0.0,1352.390137,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,2021-02-28 23:58:00,1352.390137


### Remove extraneous columns

Only these kept columns do refer to hourly information

In [11]:
weather = weather[["DATE","HourlyAltimeterSetting","HourlyDewPointTemperature","HourlyDryBulbTemperature","HourlyPrecipitation","HourlyPresentWeatherType","HourlyPressureChange","HourlyPressureTendency","HourlyRelativeHumidity","HourlySeaLevelPressure","HourlySkyConditions","HourlyStationPressure","HourlyVisibility","HourlyWetBulbTemperature","HourlyWindDirection","HourlyWindGustSpeed","HourlyWindSpeed"]]

# Clean weather data

**Considerations:**

NaN/blank cannot be assumed to be zero.

s = suspect value (appears with value)

T = trace precipitation amount or snow depth (an amount too small to measure, usually < 0.005 inches water equivalent) (appears instead of numeric value)

M = missing value (appears instead of value)

Blank = value is unreported (appears instead of value)

In [12]:
# There is a lot of NaN values in the weather data. They cannot be assumed to be zero
print(f"Size of weather: {weather.shape[0]}")
print("Number of NaN/blank values in each column:")
weather.isna().sum()

Size of weather: 67773
Number of NaN/blank values in each column:


DATE                             0
HourlyAltimeterSetting       11818
HourlyDewPointTemperature     4610
HourlyDryBulbTemperature      4610
HourlyPrecipitation          17317
HourlyPresentWeatherType     49933
HourlyPressureChange         46169
HourlyPressureTendency       46169
HourlyRelativeHumidity        4610
HourlySeaLevelPressure       16766
HourlySkyConditions           7081
HourlyStationPressure         5103
HourlyVisibility              4577
HourlyWetBulbTemperature      5105
HourlyWindDirection           4672
HourlyWindGustSpeed          58511
HourlyWindSpeed               4590
dtype: int64

### Remove redundant columns

HourlyAltimeterSetting: collinear with HourlyStationPressure

HourlyPressureChange, HourlyPressureTendency: missing many values, redundant with HourlyStationPressure

HourlySeaLevelPressure: a mapping of HourlyStationPressure.

HourlyWindGustSpeed: missing many values, redundant with HourlyWindSpeed

In [13]:
weather_clean = weather.drop(["HourlyAltimeterSetting", "HourlyPressureChange", "HourlyPressureTendency", "HourlySeaLevelPressure", "HourlyWindGustSpeed"],axis=1)

### Remove invalid rows

HourlyDryBulb is one of a set of 4 predictors that consistently occur together in most cases. Rows where they are missing are going to be missing crucial data, and should be removed to be interpolated later.

In [14]:
weather_clean = weather_clean[weather_clean["HourlyDryBulbTemperature"].isna() == False]

In [15]:
print(f"Size of weather: {weather_clean.shape[0]}")
print("Number of NaN/blank values in each column:")
weather_clean.isna().sum()

Size of weather: 63163
Number of NaN/blank values in each column:


DATE                             0
HourlyDewPointTemperature        0
HourlyDryBulbTemperature         0
HourlyPrecipitation          12710
HourlyPresentWeatherType     45323
HourlyRelativeHumidity           0
HourlySkyConditions           2472
HourlyStationPressure          495
HourlyVisibility                 1
HourlyWetBulbTemperature       495
HourlyWindDirection             96
HourlyWindSpeed                 14
dtype: int64

**Notes on remaining features**

*HourlySkyConditions*: It is likely not worth including feature in models, due to difficulty in representation. It appears to have correlation with precipitation and weather metrics, indicating it might be useful in classifying entries for interpollating missing data points.

The documentation suggests that the overall character of the sky can be classified by the topmost/final listed layer. This is how I will represent the feature.

*HourlyPresentWeatherType*: Will need to be expanded as a dummy variable, because one measurement can have multiple values. We should be sure to consider additive effects of combined weather patterns.

Each weather item is indicated to be light, moderate, and heavy. For now, I will avoid including this information. It may be worth expanding this info to additonal dummy variables if some weather type is shown to have usefulness.

*HourlyPrecipitation*: Need to convert T (trace amount) to a number. I will use 0.01, as that seems to be used interchangably in this data set.

### Reduce HourlySkyConditions
Coverage: CLR (clear sky), FEW (few clouds), SCT (scattered clouds), BKN (broken clouds), OVC (overcast), VV (obscured sky)

In [16]:
def reduceSkyConditions(conditions):
    condition = conditions.split(" ")
    if len(condition) >= 2:
        condition = condition[-2]  # Take last condition ([-1] would be height of last condition)
        condition = condition.split(":")[0]
        return str(condition)
    elif len(condition) == 1:
        if condition[0] == "CLR:00":
            return "CLR"
        else:
            return np.NaN
    else:
        return np.NaN

weather_clean["HourlySkyConditions"] = weather_clean.apply(lambda x: reduceSkyConditions(str(x["HourlySkyConditions"])), axis=1)
weather_clean["HourlySkyConditions"].unique()

array(['BKN', nan, 'OVC', 'VV', 'SCT', 'FEW', 'CLR', 'X'], dtype=object)

### Reduce present weather type

Three values given, seperated by pipes. The first (AU, automatic sensor) seems to be most valid for this data set.

In [17]:
# Identify all used AU Codes

au_codes = []

def findAUCodes(conditions):
    condition = conditions.split("|")
    if len(condition) > 1:
        condition = condition[0]
        condition = condition.split(" ")
        types = []
        for cond in condition:
            temp = cond.split(":")[0]
#            if temp[0] == '-' or temp[0] == '+':
#                temp = temp[1:]
            types.append(temp)
        return types
    else:
        return None

out = weather.apply(lambda x: findAUCodes(str(x["HourlyPresentWeatherType"])), axis=1)

for i in out:
    if i is not None:
        for j in i:
            au_codes.append(j)

set(au_codes)

{'',
 '+PL',
 '+RA',
 '+SN',
 '-DZ',
 '-FZ',
 '-GS',
 '-PL',
 '-RA',
 '-SN',
 '-TS',
 'BC',
 'BL',
 'BR',
 'DZ',
 'FG',
 'FZ',
 'GR',
 'GS',
 'HZ',
 'MI',
 'PL',
 'RA',
 'SN',
 'SQ',
 'TS',
 'UP',
 'VCTS'}

In [18]:
#include possible AU codes from above block (option to ignore +/-)
au_codes =  [
                'BC',
                'BL',
                'BR',
                'DZ',
                'FG',
                'FZ',
                'GS',
                'HZ',
                'MI',
                'PL',
                'RA',
                'SN',
                'TS',
                'VCTS'  # VCTS will co-occur with TS by this code, should be okay though. Can be improved with assumption that VCTS and TS are mutually exclusive
            ]

def reduceWeatherTypes(conditions):
    condition = conditions.split("|")
    if len(condition) > 1:
        condition = condition[0]
        au_dummy = []
        for au in au_codes:
            au_dummy.append(int(au in condition))
        return au_dummy
    else:
        return [0] * len(au_codes)
    
weather_type_dummies = weather_clean.apply(lambda x: reduceWeatherTypes(str(x["HourlyPresentWeatherType"])), result_type='expand', axis=1)
weather_type_dummies.columns = au_codes
# weather_clean = weather_clean.drop(["HourlyPresentWeatherType"],axis=1)  # Should be done after NaN's are dealt with
weather_clean = pd.concat([weather_clean, weather_type_dummies], axis=1)

### Replace T with number in HourlyPrecipitation

In [19]:
weather_clean.loc[weather_clean.HourlyPrecipitation == 'T', 'HourlyPrecipitation'] = 0.01

# Create hourly summary data

There's probably a ton of ways to do this. I will leverage here the observation that the measurements at 54 minute marks are the most consistently good. I will then map each of these hourly values to the nearest whole hour.

In [20]:
weather_summ = weather_clean[weather_clean["DATE"].dt.minute == 54]

In [21]:
print(f"Size of weather: {weather_summ.shape[0]}")
print("Number of NaN/blank values in each column:")
weather_summ.isna().sum()

Size of weather: 43789
Number of NaN/blank values in each column:


DATE                             0
HourlyDewPointTemperature        0
HourlyDryBulbTemperature         0
HourlyPrecipitation            399
HourlyPresentWeatherType     35171
HourlyRelativeHumidity           0
HourlySkyConditions            336
HourlyStationPressure          333
HourlyVisibility                 1
HourlyWetBulbTemperature       333
HourlyWindDirection              9
HourlyWindSpeed                  9
BC                               0
BL                               0
BR                               0
DZ                               0
FG                               0
FZ                               0
GS                               0
HZ                               0
MI                               0
PL                               0
RA                               0
SN                               0
TS                               0
VCTS                             0
dtype: int64

##### Assumptions to clean summary data:

Many NaN around 2019-11-22. Maintenance? 

HourlyPrecipitation needs further investigation. Must beware T (Trace) values. Appears to be similar gaps to HourlySkyConditions

HourlySkyConditions have gaps of missing data. Perhaps use a classifier to predict for missing values.

In [22]:
# Adjust each datetime to the nearest hour ***ROUNDING DOWN***
weather_summ["DATE"] = weather_summ["DATE"].apply(lambda x: x.replace(minute=0))
#weather_summ.set_index("DATE")

C:\Users\Wade\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [23]:
all = pd.Series(data=pd.date_range(start=weather_summ["DATE"].min(), end=weather_summ["DATE"].max(), freq='H'))
mask = all.isin(weather_summ["DATE"].values)
print(all[~mask])

3960    2016-08-13 00:00:00
5617    2016-10-21 01:00:00
5618    2016-10-21 02:00:00
5619    2016-10-21 03:00:00
5620    2016-10-21 04:00:00
5621    2016-10-21 05:00:00
6943    2016-12-15 07:00:00
6986    2016-12-17 02:00:00
14742   2017-11-05 06:00:00
14978   2017-11-15 02:00:00
15097   2017-11-20 01:00:00
15677   2017-12-14 05:00:00
15888   2017-12-23 00:00:00
23761   2018-11-16 01:00:00
25275   2019-01-18 03:00:00
25493   2019-01-27 05:00:00
26024   2019-02-18 08:00:00
27013   2019-03-31 13:00:00
30774   2019-09-04 06:00:00
31917   2019-10-21 21:00:00
32425   2019-11-12 01:00:00
32677   2019-11-22 13:00:00
32679   2019-11-22 15:00:00
32948   2019-12-03 20:00:00
32949   2019-12-03 21:00:00
33117   2019-12-10 21:00:00
33118   2019-12-10 22:00:00
33119   2019-12-10 23:00:00
33128   2019-12-11 08:00:00
33134   2019-12-11 14:00:00
33135   2019-12-11 15:00:00
33187   2019-12-13 19:00:00
33188   2019-12-13 20:00:00
33212   2019-12-14 20:00:00
39110   2020-08-16 14:00:00
dtype: datetime64[ns

We need to fill in the missing data points. This can probably be done by using the averages of the other data points nearby.

# Cleaning power data

It seems that some engines tend to show negative numbers consistently or during startup. This needs to be further investigated.

Otherwise, power data can be converted to hourly information either by an average of all measurements in an hour or by sampling at regular hour intervals. Both methods are valid and produce similar results. The biggest proportional difference occurs in the small (sub 5000 Tonnes) power ranges. Ways to calculate each and compare in Excel are given below.

There might be appliations where a max or min value over an hour will be useful as well.

```=AVERAGE(OFFSET(power!$K$2,(ROW(A1)-1)*60,,60,))```

```=OFFSET(power!$K$2,(ROW(B1)-1)*60,,1,)```

```=(B1-A1)/B1```

In [24]:
# No cleaning done yet
power_clean = power[["DATE","Total Power"]]
#power_clean = power.set_index(["DATE"])

In [25]:
# Check for time series continuity
all_date = pd.Series(data=pd.date_range(start=power_clean["DATE"].min(), end=power_clean["DATE"].max(), freq='H'))
mask = all_date.isin(power_clean["DATE"].values)
print(all_date[~mask])
missing_dates = all_date[~mask].to_list()

290     2016-03-13 02:00:00
9026    2017-03-12 02:00:00
17762   2018-03-11 02:00:00
26498   2019-03-10 02:00:00
35234   2020-03-08 02:00:00
dtype: datetime64[ns]


In [26]:
# This value should be zero to be consistent with surrounding data values
missing_rows = [ [x, power_clean[power_clean['DATE'] == (x-pd.Timedelta(1, unit="h"))]['Total Power'].values[0]] 
                for x in missing_dates]
missing_val = pd.DataFrame(missing_rows, columns=['DATE','Total Power'])
power_clean = power_clean.append(missing_val, ignore_index=True)

power_clean = power_clean.sort_values(['DATE'])
power_clean = power_clean.reset_index(drop=True)
#power_clean = power_clean[:-1]

In [27]:
power_clean

,DATE,Total Power
0,2016-03-01 00:00:00,1179.439331
1,2016-03-01 00:01:00,1195.945435
2,2016-03-01 00:02:00,1175.737549
3,2016-03-01 00:03:00,1179.982910
4,2016-03-01 00:04:00,1198.777954
...,...,...
2629440,2021-02-28 23:55:00,1382.180176
2629441,2021-02-28 23:56:00,1367.581299
2629442,2021-02-28 23:57:00,1352.855835
2629443,2021-02-28 23:58:00,1352.390137


In [28]:
# Check for time series continuity
all_date = pd.Series(data=pd.date_range(start=power_clean["DATE"].min(), end=power_clean["DATE"].max(), freq='H'))
mask = all_date.isin(power_clean["DATE"].values)
print(all_date[~mask])

Series([], dtype: datetime64[ns])


### Convert to hourly summary

Min and Max need to be fixed, currently they match all rows with min/max values, should only match those with right time and min/max

In [29]:
power_summ_all = power_clean[power_clean['DATE'].dt.minute == 30][['DATE']]
power_summ_all["DATE"] = power_summ_all["DATE"].apply(lambda x: x.replace(minute=0))

In [30]:
power_summ_max = power_clean.groupby([power_clean["DATE"].dt.year, power_clean["DATE"].dt.dayofyear, power_clean["DATE"].dt.hour], sort=False)['Total Power'].max().tolist()
power_summ_all['Total Power (max)'] = power_summ_max

In [31]:
power_summ_min = power_clean.groupby([power_clean["DATE"].dt.year, power_clean["DATE"].dt.dayofyear, power_clean["DATE"].dt.hour], sort=False)['Total Power'].min().tolist()
power_summ_all['Total Power (min)'] = power_summ_min

In [32]:
power_summ_avg = power_clean.groupby([power_clean["DATE"].dt.year, power_clean["DATE"].dt.dayofyear, power_clean["DATE"].dt.hour], sort=False)['Total Power'].mean().tolist()
power_summ_all['Total Power (avg)'] = power_summ_avg

In [33]:
# Get regular interval hourly (on top of the hour)
power_summ_idx = power_clean['DATE'].dt.minute == 54
power_summ_samp = power_clean[power_summ_idx]['Total Power'].tolist()
power_summ_all['Total Power (samp)'] = power_summ_samp

In [34]:
# TRUE: 1.5*IQR bound
# FALSE: trim at 0.95
if False:
    # find upper bound for an hour's power and cap max value accordingly
    grouped = power_clean.groupby([power_clean["DATE"].dt.year, power_clean["DATE"].dt.dayofyear, power_clean["DATE"].dt.hour], sort=False)['Total Power']
    q3 = grouped.quantile(0.75)
    q1 = grouped.quantile(0.25)
    capmax = (2.5*q3) - (1.5*q1)
    s = grouped.size().tolist()
    allmax = [item for item, count in zip(capmax, s) for i in range(count)]

    power_summ_capmax = power_clean[power_clean['Total Power'] <= pd.Series(allmax)].groupby([power_clean["DATE"].dt.year, power_clean["DATE"].dt.dayofyear, power_clean["DATE"].dt.hour], sort=False)['Total Power'].max().tolist()
    power_summ_all['Total Power (max w/o outliers)'] = power_summ_capmax
else:
    power_summ_capmax = power_clean.groupby([power_clean["DATE"].dt.year, power_clean["DATE"].dt.dayofyear, power_clean["DATE"].dt.hour], sort=False)['Total Power'].quantile(0.95).tolist()
    power_summ_all['Total Power (trimmed)'] = power_summ_capmax

# Merge data sets

In [35]:
data = pd.merge(power_summ_all, weather_summ, how="left", on='DATE')

# Interpolate missing data
Temporary solution for now, fill in NaN rows with row above

In [36]:
# First, column HourlyPresentWeatherType can be removed, if nothin is present, if should be clear weather
data = data.drop(['HourlyPresentWeatherType'],axis=1)

In [37]:
# Some numeric values have a trailing 's' in their data type
cols = ['HourlyDewPointTemperature', 'HourlyDryBulbTemperature',
       'HourlyPrecipitation', 'HourlyRelativeHumidity',
       'HourlyStationPressure', 'HourlyVisibility', 'HourlyWetBulbTemperature',
       'HourlyWindDirection', 'HourlyWindSpeed']
for col in cols:
    data[col] = data[col].astype(str).str.extract(r'(\d+\.\d+|\d+)', expand=False).astype(float)

In [38]:
print(f"Size of data: {data.shape[0]}")
print("Number of NaN/blank values in each column:")
data.isna().sum()

Size of data: 43824
Number of NaN/blank values in each column:


DATE                           0
Total Power (max)              0
Total Power (min)              0
Total Power (avg)              0
Total Power (samp)             0
Total Power (trimmed)          0
HourlyDewPointTemperature     35
HourlyDryBulbTemperature      35
HourlyPrecipitation          434
HourlyRelativeHumidity        35
HourlySkyConditions          371
HourlyStationPressure        368
HourlyVisibility              36
HourlyWetBulbTemperature     368
HourlyWindDirection          772
HourlyWindSpeed               44
BC                            35
BL                            35
BR                            35
DZ                            35
FG                            35
FZ                            35
GS                            35
HZ                            35
MI                            35
PL                            35
RA                            35
SN                            35
TS                            35
VCTS                          35
dtype: int

In [39]:
print(data[data.HourlyWetBulbTemperature.isna()])

                     DATE  Total Power (max)  Total Power (min)  \
3959  2016-08-13 00:00:00       16819.509644       15859.861695   
5525  2016-10-17 06:00:00        9830.459105        9147.666992   
5616  2016-10-21 01:00:00        4637.665649        4240.743042   
5617  2016-10-21 02:00:00        4405.222107        3957.757569   
5618  2016-10-21 03:00:00        4140.148072        3813.323608   
...                   ...                ...                ...   
33253 2019-12-16 13:00:00        1190.562500        1105.034790   
34665 2020-02-13 09:00:00           0.000000           0.000000   
38710 2020-07-30 23:00:00       12805.599121       12381.304199   
39109 2020-08-16 14:00:00       17051.412109       15738.970947   
43127 2021-01-30 23:00:00        1725.273071        1620.433472   

       Total Power (avg)  Total Power (samp)  Total Power (trimmed)  \
3959        16259.604106        15833.288817           16703.026178   
5525         9579.900191         9875.745240         

In [40]:
data = data.fillna(method='ffill')

### Export to csv

In [41]:
data.to_csv("merged_data_5yr.csv")